In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
from etrace.load_data import load_from_bq

## Import Data form BigQuery using load_from_bq

In [4]:
df = load_from_bq("SELECT * FROM aklewagonproject.etrace.cleaned_final_interpolated_dataset")

/Users/jaumerayo/.pyenv/versions/3.12.9/envs/etrace_env/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [5]:
df.sort_values(['geo','year'], ascending=True)

# Handling missing data through interpolation
df['gdp_capita'] = df['gdp_capita'].interpolate(method='linear', limit_direction='forward')
df['pop'] = df['pop'].interpolate(method='linear', limit_direction='forward')
df['employment_rate'] = df['employment_rate'].interpolate(method='linear', limit_direction='forward')

counts = df["geo"].value_counts()

print('Unique geo values:', len(df['geo'].unique()))
print(counts)

Unique geo values: 282
geo
ES52    21
ES53    21
ES24    21
PT11    21
ES41    21
        ..
FRI3     1
RS22     1
FRK1     1
FRY2     1
FRY1     1
Name: count, Length: 282, dtype: int64


We can see that we have some NUTS-2 IDs with just 1 unit of value, to solve it and allow the model to generalize it lets delate it.

In [6]:
nuts2_counts = df.geo.value_counts()

#minimum threshold
tol_value = 5

nuts2_filter = nuts2_counts[nuts2_counts  >= tol_value].index

#filter df, we could also use .apply with lambda function
df_filtered = df[df['geo'].isin(nuts2_filter)]

Quick check to see how the dataframe changes

In [7]:
print(f"Original: {len(df)} rows with {df['geo'].nunique()} regions")
print(f"Filtered: {len(df_filtered)} rows with {df_filtered['geo'].nunique()} regions")
print(f"Regions removed: {df['geo'].nunique() - df_filtered['geo'].nunique()}")

Original: 4317 rows with 282 regions
Filtered: 4196 rows with 236 regions
Regions removed: 46


In [8]:
df_filtered.head()

,geo,year,pop,employment_rate,gdp_capita,NUTS_NAME,area_km2,pop_dens,pct_Dfb,pct_Dfc,...,pct_Cfc,pct_BWh,pct_Af,pct_Am,pct_Aw,pct_Cwa,pct_Cwb,pct_Csc,pct_Dsa,nights_spent
0,ES52,2000,4103816.0,62.5,15200.0,Comunitat Valenciana,23264.016726,176.401868,0.000015,0.0,...,0.0,0.058283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32567572.0
1,ES52,2001,4135183.0,63.9,16500.0,Comunitat Valenciana,23264.016726,177.750173,0.000012,0.0,...,0.0,0.059404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33505671.0
2,ES52,2002,4192277.0,64.5,17200.0,Comunitat Valenciana,23264.016726,180.204349,0.000009,0.0,...,0.0,0.060528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33564801.0
3,ES52,2003,4322066.0,64.9,17900.0,Comunitat Valenciana,23264.016726,185.783309,0.000006,0.0,...,0.0,0.061654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34536607.0
4,ES52,2004,4441941.0,66.7,18600.0,Comunitat Valenciana,23264.016726,190.936116,0.000003,0.0,...,0.0,0.062782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35862548.0


In [9]:
# Define columns
cols_climate = [c for c in df_filtered.columns if c.startswith("pct_")]
cols_econ = ["year", "pop", "employment_rate", "gdp_capita", "pop_dens"]
cols_cat = ['geo']

In [10]:
features = cols_cat + cols_econ + cols_climate
target = 'nights_spent'

In [11]:
# Separate categorical and numerical columns
cat_cols = cols_cat
num_cols = cols_econ + cols_climate

### RobustScaler chosen over StandardScaler due to significant outliers in economic variables
### OneHotEncoder for categorical 'geo' variable

In [12]:
print(df_filtered[cols_econ + cols_climate].describe())

              year           pop  employment_rate     gdp_capita     pop_dens  \
count       4196.0  4.196000e+03      4196.000000    4196.000000  4124.000000   
mean   2010.595806  1.839992e+06        68.801656   24568.112488   319.346071   
std       6.030089  1.679040e+06         8.918510   14605.344080   775.035664   
min         2000.0  2.570600e+04        32.800000    1400.000000     3.072921   
25%         2005.0  8.442932e+05        63.075000   12800.000000    70.009652   
50%         2011.0  1.360918e+06        70.200000   23800.000000   118.379963   
75%         2016.0  2.190951e+06        75.700000   33500.000000   238.280985   
max         2020.0  1.551927e+07        88.400000  102200.000000  7562.137509   

           pct_Dfb      pct_Dfc       pct_ET       pct_EF      pct_Cfb  ...  \
count  4196.000000  4196.000000  4196.000000  4196.000000  4196.000000  ...   
mean      0.239799     0.048563     0.008694     0.000011     0.368047  ...   
std       0.346435     0.157607  

In [13]:
# Preprocessor, diferent for each class
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ("num", RobustScaler(), num_cols)
])

In [14]:
# Model
model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)


In [15]:
# Pipeline
pipeline = Pipeline([
    ("prep", preprocessor),
    ("xgb", model)
])

# Store results per region
results_dict = {}

In [16]:
# Train model for each region
for region, group in df_filtered.groupby("geo"):

    # Split temporal using int year
    train = group[group["year"] <= 2018]
    test = group[group["year"] > 2018]

    # Validate sufficient data
    if len(train) < 2 or len(test) < 2:
        print(f"Geo {region} has little data and is omitted")
        continue

    # Split data for train and test
    X_train = train[features]
    y_train = train[target]
    X_test = test[features]
    y_test = test[target]

    # Train
    pipeline.fit(X_train, y_train)

    # Predict and calculate metrics
    y_pred = pipeline.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Normalized Mean Absolute Error
    mean_y = y_test.mean()
    nmae = mae / mean_y

    # Save pipeline and metrics
    results_dict[region] = {
        "pipeline": pipeline,
        "mae": mae,
        "rmse": rmse,
        "mean_y": mean_y,
        "nmae": nmae
    }

# Display performance
print("\n# Performance Summary")
for region, res in results_dict.items():
    print(f"{region}: MAE={res['mae']:.2f}, RMSE={res['rmse']:.2f}, NMAE={res['nmae']:.2f}")

Geo CH01 has little data and is omitted
Geo CH02 has little data and is omitted
Geo CH03 has little data and is omitted
Geo CH04 has little data and is omitted
Geo CH05 has little data and is omitted
Geo CH06 has little data and is omitted
Geo CH07 has little data and is omitted
Geo ME00 has little data and is omitted
Geo NO02 has little data and is omitted
Geo NO06 has little data and is omitted
Geo NO07 has little data and is omitted

# Performance Summary
AT11: MAE=408975.25, RMSE=527842.10, NMAE=0.17
AT12: MAE=1464997.75, RMSE=1915349.29, NMAE=0.27
AT13: MAE=6195799.50, RMSE=8102479.34, NMAE=0.59
AT21: MAE=1022043.00, RMSE=1431634.78, NMAE=0.10
AT22: MAE=1521967.50, RMSE=2016615.12, NMAE=0.15
AT31: MAE=1423625.50, RMSE=1809334.27, NMAE=0.23
AT32: MAE=4238671.00, RMSE=5639298.63, NMAE=0.21
AT33: MAE=6776314.00, RMSE=9379343.64, NMAE=0.21
AT34: MAE=1169114.00, RMSE=1595211.02, NMAE=0.19
BE10: MAE=2804274.50, RMSE=3670632.00, NMAE=0.61
BE21: MAE=1427593.50, RMSE=1985410.80, NMAE=0.41


# Comparasion between two different approach

In [20]:
print("="*70)
print("APPROACH 1: SEPARATE MODELS (one per region)")
print("="*70)

# store results per region
results_separate = {}

# train model for each region
for region, group in df_filtered.groupby("geo"):

    # temporal
    train = group[group["year"] <= 2018]
    test = group[group["year"] > 2018]

    # Validate sufficient data
    if len(train) < 2 or len(test) < 2:
        continue

    # Split data
    X_train = train[features]
    y_train = train[target]
    X_test = test[features]
    y_test = test[target]

    # Pipeline
    pipeline = Pipeline([
        ("prep", preprocessor),
        ("xgb", model)
    ])

    # Train
    pipeline.fit(X_train, y_train)

    # Predict and calculate metrics
    y_pred = pipeline.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_y = y_test.mean()
    nmae = mae / mean_y

    results_separate[region] = {
        "mae": mae,
        "rmse": rmse,
        "nmae": nmae,
        "n_test": len(y_test)
    }

# Convert to DataFrame
df_separate = pd.DataFrame(results_separate).T
df_separate.index.name = 'region'
df_separate = df_separate.reset_index()

print(f"\nRegions trained: {len(df_separate)}")
print(f"Mean NMAE: {df_separate['nmae'].mean():.3f}")
print(f"Median NMAE: {df_separate['nmae'].median():.3f}")
print(f"Std NMAE: {df_separate['nmae'].std():.3f}")

print("\n" + "="*70)
print("APPROACH 2: GLOBAL MODEL (all regions together)")
print("="*70)

# Split temporal for ALL regions together
train = df_filtered[df_filtered["year"] <= 2018]
test = df_filtered[df_filtered["year"] > 2018]

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

print(f"\nTrain set: {len(X_train)} observations")
print(f"Test set: {len(X_test)} observations")

# Pipeline
pipeline_global = Pipeline([
    ("prep", preprocessor),
    ("xgb", model)
])

# Train
pipeline_global.fit(X_train, y_train)

# Predict
y_pred_global = pipeline_global.predict(X_test)

# Global metrics
mae_global = mean_absolute_error(y_test, y_pred_global)
rmse_global = np.sqrt(mean_squared_error(y_test, y_pred_global))
mean_y_global = y_test.mean()
nmae_global = mae_global / mean_y_global

print(f"\nGlobal Performance:")
print(f"MAE: {mae_global:,.2f}")
print(f"RMSE: {rmse_global:,.2f}")
print(f"NMAE: {nmae_global:.3f}")

# Performance by region for global model
results_global = []
test_with_pred = test.copy()
test_with_pred['y_pred'] = y_pred_global

for region in df_filtered['geo'].unique():
    region_data = test_with_pred[test_with_pred['geo'] == region]

    if len(region_data) > 0:
        y_test_region = region_data[target]
        y_pred_region = region_data['y_pred']

        mae_region = mean_absolute_error(y_test_region, y_pred_region)
        rmse_region = np.sqrt(mean_squared_error(y_test_region, y_pred_region))
        mean_y_region = y_test_region.mean()
        nmae_region = mae_region / mean_y_region

        results_global.append({
            'region': region,
            'mae': mae_region,
            'rmse': rmse_region,
            'nmae': nmae_region,
            'n_test': len(region_data)
        })

df_global = pd.DataFrame(results_global)

print(f"\nRegions evaluated: {len(df_global)}")
print(f"Mean NMAE by region: {df_global['nmae'].mean():.3f}")
print(f"Median NMAE by region: {df_global['nmae'].median():.3f}")
print(f"Std NMAE by region: {df_global['nmae'].std():.3f}")

APPROACH 1: SEPARATE MODELS (one per region)

Regions trained: 225
Mean NMAE: 0.309
Median NMAE: 0.285
Std NMAE: 0.144

APPROACH 2: GLOBAL MODEL (all regions together)

Train set: 3736 observations
Test set: 460 observations

Global Performance:
MAE: 3,502,160.67
RMSE: 8,087,055.46
NMAE: 0.411

Regions evaluated: 235
Mean NMAE by region: 0.764
Median NMAE by region: 0.390
Std NMAE by region: 2.835

COMPARISON

Total regions compared: 225

Global model wins: 60 regions
Separate models win: 165 regions

Average NMAE difference (Separate - Global): -0.4717
Median NMAE difference: -0.0251

Top 10 regions where Global model is BETTER:
    region  nmae_separate  nmae_global  nmae_diff
86    EL54       0.535508     0.318252   0.217257
176   PT30       0.570201     0.411881   0.158319
199   TR10       0.402502     0.291467   0.111035
203   TR32       0.400386     0.307152   0.093234
87    EL61       0.469136     0.388554   0.080582
217   TR90       0.284239     0.206011   0.078228
92    ES11  

In [ ]:
print("\n" + "="*70)
print("COMPARISON")
print("="*70)

# Merge results for comparison
df_comparison = df_separate.merge(
    df_global,
    on='region',
    suffixes=('_separate', '_global')
)

df_comparison['nmae_diff'] = df_comparison['nmae_separate'] - df_comparison['nmae_global']
df_comparison['winner'] = df_comparison['nmae_diff'].apply(
    lambda x: 'Global' if x > 0 else 'Separate'
)

print(f"\nTotal regions compared: {len(df_comparison)}")
print(f"\nGlobal model wins: {(df_comparison['winner'] == 'Global').sum()} regions")
print(f"Separate models win: {(df_comparison['winner'] == 'Separate').sum()} regions")

print(f"\nAverage NMAE difference (Separate - Global): {df_comparison['nmae_diff'].mean():.4f}")
print(f"Median NMAE difference: {df_comparison['nmae_diff'].median():.4f}")

print("\nTop 10 regions where Global model is BETTER:")
print(df_comparison.nlargest(10, 'nmae_diff')[['region', 'nmae_separate', 'nmae_global', 'nmae_diff']])

print("\nTop 10 regions where Separate model is BETTER:")
print(df_comparison.nsmallest(10, 'nmae_diff')[['region', 'nmae_separate', 'nmae_global', 'nmae_diff']])


COMPARISON

Total regions compared: 225

Global model wins: 60 regions
Separate models win: 165 regions

Average NMAE difference (Separate - Global): -0.4717
Median NMAE difference: -0.0251

Top 10 regions where Global model is BETTER:
    region  nmae_separate  nmae_global  nmae_diff
86    EL54       0.535508     0.318252   0.217257
176   PT30       0.570201     0.411881   0.158319
199   TR10       0.402502     0.291467   0.111035
203   TR32       0.400386     0.307152   0.093234
87    EL61       0.469136     0.388554   0.080582
217   TR90       0.284239     0.206011   0.078228
92    ES11       0.390449     0.313516   0.076933
82    EL43       0.658997     0.586792   0.072204
207   TR51       0.292494     0.222006   0.070488
204   TR33       0.236405     0.171561   0.064844

Top 10 regions where Separate model is BETTER:
    region  nmae_separate  nmae_global  nmae_diff
109   ES64       0.471783    42.574949 -42.103166
108   ES63       0.515340     7.277242  -6.761902
131   ITF2     

## Model Comparison Results

We tested two approaches:
1. **Separate models**: One model per region (225 models)
2. **Global model**: Single model for all regions

### Performance

The separate models clearly outperformed the global approach:
- Separate models: **31% average error**
- Global model: **76% average error**

Separate models won in **73% of regions** (165 out of 225).

### Key Finding

The global model had catastrophic failures in several Eastern European regions (Poland, Romania, Greece), with prediction errors exceeding 200%. This suggests that each region has unique tourism patterns that can't be adequately captured by a one-size-fits-all approach.

### Conclusion

We'll proceed with the **separate models approach** for final predictions, as it provides more accurate and reliable results across all regions.